# Import Libraries

In [8]:
import cv2
import os
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm
import logging

# This code resize by skipping pixels in the middle

In [76]:
# 1 
class ImageResizer(BaseEstimator, TransformerMixin):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [cv2.resize(img, (self.width, self.height), interpolation=cv2.INTER_AREA) for img in X]


# 2
class CLAHEEnhancer(BaseEstimator, TransformerMixin):
    def __init__(self, clip_limit=2.0, tile_grid_size=(16, 16)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        return [clahe.apply(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in X]


def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))

def preprocess_images(image_folder, output_dir):
    logging.basicConfig(filename='image_processing.log', level=logging.INFO)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    processed_dir = os.path.join(output_dir, 'processed_images(Clache+size)')
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)
    
    # Create the pipeline
    pipeline = make_pipeline(
        ImageResizer(width=1935, height=1024),
        CLAHEEnhancer(clip_limit=2.0, tile_grid_size=(16, 16))
    )
    
    # Get a list of image files in the folder
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if is_image_file(file)]
    
    for image_path in tqdm(image_files, desc="Processing images"):
        try:
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)
            if img is None:
                raise ValueError(f"Image is None: {image_path}")
            
            # Apply the pipeline
            processed_img = pipeline.transform([img])[0]
            
            output_path = os.path.join(processed_dir, os.path.basename(image_path))
            cv2.imwrite(output_path, processed_img)
        except Exception as e:
            logging.error(f"Error processing image {image_path}: {str(e)}")

# Folder containing your X-ray images (adjust this path)
image_folder_path = r"C:\Users\hp\Desktop\hepsi\ONCE\once-27"
# Directory where you want to save processed images
processed_images_dir = r"C:\Users\hp\Desktop\hepsi\ONCE\once-27-27"
# Call the function with the image folder path and processed directory
preprocess_images(image_folder_path, processed_images_dir)

Processing images: 100%|██████████| 30/30 [00:01<00:00, 18.54it/s]


# Image individually

In [27]:
import os
import cv2
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline

class ImageResizer(BaseEstimator, TransformerMixin):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [cv2.resize(img, (self.width, self.height), interpolation=cv2.INTER_AREA) for img in X]

class CLAHEEnhancer(BaseEstimator, TransformerMixin):
    def __init__(self, clip_limit=2.0, tile_grid_size=(16, 16)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        return [clahe.apply(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in X]

def process_image(image_path, output_dir):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"Failed to load image: {image_path}")
            return False

        pipeline = make_pipeline(
            ImageResizer(width=1935, height=1024),
            CLAHEEnhancer(clip_limit=2.0, tile_grid_size=(16, 16))
        )

        processed_img = pipeline.transform([img])[0]
        
        output_filename = os.path.basename(image_path)
        output_path = os.path.join(output_dir, output_filename)
        cv2.imwrite(output_path, processed_img)
        print(f"Processed and saved: {output_path}")
        return True
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return False

def main():
    # Specify the input image path here
    input_path = r"C:\Users\hp\Desktop\hepsi\gümülü\gumulu-1\ABDULKARIM_ANADANI_jpg_ca93bbbd-5f1a-47c1-b087-72017a42fa11.jpg"
    
    # Specify the output directory here
    output_dir = r"C:\Users\hp\Desktop\hepsi\gümülü\gumulu-1-1\ABDULKARIM_ANADANI_jpg_ca93bbbd-5f1a-47c1-b087-72017a42fa11.jpg"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    success = process_image(input_path, output_dir)
    if success:
        print("Image processing completed successfully.")
    else:
        print("Image processing failed.")

if __name__ == "__main__":
    main()

Failed to load image: C:\Users\hp\Desktop\hepsi\gümülü\gumulu-1\ABDULKARIM_ANADANI_jpg_ca93bbbd-5f1a-47c1-b087-72017a42fa11.jpg
Image processing failed.


# 1 is the resizer
# 2 is the CLACHE

# This Code resize by edges around the image

In [30]:
# import os
# import cv2
# import logging
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import make_pipeline

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# class ImageCropper(BaseEstimator, TransformerMixin):
#     def __init__(self, width, height):
#         """
#         Initialize the ImageCropper with the desired width and height.
#         """
#         self.width = width
#         self.height = height
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         """
#         Crop images to the specified width and height, keeping the center part.
#         If the image is smaller than the target size, resize it first.
#         """
#         cropped_images = []
#         for img in X:
#             if img.shape[1] < self.width or img.shape[0] < self.height:
#                 # Resize the image to the minimum required dimensions
#                 img = cv2.resize(img, (max(self.width, img.shape[1]), max(self.height, img.shape[0])), interpolation=cv2.INTER_AREA)
            
#             center_x, center_y = img.shape[1] // 2, img.shape[0] // 2
#             half_width, half_height = self.width // 2, self.height // 2
#             cropped_img = img[center_y - half_height:center_y + half_height, center_x - half_width:center_x + half_width]
#             cropped_images.append(cropped_img)
#         return cropped_images


# class CLAHEEnhancer(BaseEstimator, TransformerMixin):
#     def __init__(self, clip_limit=2.0, tile_grid_size=(16, 16)):
#         """
#         Initialize the CLAHEEnhancer with the desired clip limit and tile grid size.
#         """
#         self.clip_limit = clip_limit
#         self.tile_grid_size = tile_grid_size
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         """
#         Apply CLAHE to enhance the contrast of images.
#         """
#         clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
#         return [clahe.apply(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in X]

# def preprocess_images(image_folder, output_dir):
#     """
#     Preprocess images by cropping and applying CLAHE, then save the processed images.
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     processed_dir = os.path.join(output_dir, 'processed_images(Clache+size)')
#     if not os.path.exists(processed_dir):
#         os.makedirs(processed_dir)
    
#     # Create the pipeline
#     pipeline = make_pipeline(
#         ImageCropper(width=1930, height=1000),
#         CLAHEEnhancer(clip_limit=2.0, tile_grid_size=(16, 16))
#     )
    
#     # Get a list of image files in the folder
#     image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder)]
    
#     for image_path in image_files:
#         try:
#             img = cv2.imread(image_path, cv2.IMREAD_COLOR)
#             if img is not None:
#                 # Apply the pipeline
#                 processed_img = pipeline.transform([img])[0]
                
#                 output_path = os.path.join(processed_dir, os.path.basename(image_path))
#                 cv2.imwrite(output_path, processed_img)
#                 logging.info(f"Processed and saved image: {output_path}")
#             else:
#                 logging.error(f"Failed to load image: {image_path}")
#         except Exception as e:
#             logging.error(f"Error processing image {image_path}: {e}")

# # Folder containing your X-ray images (adjust this path)
# image_folder_path = r"C:\Users\hp\Desktop\2914---1470"
# # Directory where you want to save processed images
# processed_images_dir = r"C:\Users\hp\Desktop\ProcessedImages"

# # Call the function with the image folder path and processed directory
# preprocess_images(image_folder_path, processed_images_dir)


# Eggy shape

In [36]:
# import os
# import cv2
# import logging
# import numpy as np
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.pipeline import make_pipeline

# # Set up logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# class EggShapeCropper(BaseEstimator, TransformerMixin):
#     def __init__(self, width, height, axes_ratio=(1, 1)):
#         """
#         Initialize the EggShapeCropper with the desired width, height, and axes ratio.
#         """
#         self.width = width
#         self.height = height
#         self.axes_ratio = axes_ratio  # Ratio to control the shape of the egg

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         """
#         Crop images to the specified egg shape, keeping the center part.
#         """
#         cropped_images = []
#         for img in X:
#             # Ensure the image is large enough
#             if img.shape[1] < self.width or img.shape[0] < self.height:
#                 img = cv2.resize(img, (max(self.width, img.shape[1]), max(self.height, img.shape[0])), interpolation=cv2.INTER_AREA)
            
#             # Create an egg-shaped mask
#             mask = np.zeros((self.height, self.width), dtype=np.uint8)
#             center_x, center_y = self.width // 2, self.height // 2
#             axes = (int(self.width // 2 * self.axes_ratio[0]), int(self.height // 2 * self.axes_ratio[1]))  # Adjust the axes here
#             cv2.ellipse(mask, (center_x, center_y), axes, 0, 0, 360, 255, -1)
            
#             # Crop the image using the mask
#             center_x_img, center_y_img = img.shape[1] // 2, img.shape[0] // 2
#             half_width, half_height = self.width // 2, self.height // 2
#             cropped_img = img[center_y_img - half_height:center_y_img + half_height, center_x_img - half_width:center_x_img + half_width]
#             cropped_img = cv2.bitwise_and(cropped_img, cropped_img, mask=mask)
            
#             cropped_images.append(cropped_img)
#         return cropped_images

# class CLAHEEnhancer(BaseEstimator, TransformerMixin):
#     def __init__(self, clip_limit=2.0, tile_grid_size=(16, 16)):
#         """
#         Initialize the CLAHEEnhancer with the desired clip limit and tile grid size.
#         """
#         self.clip_limit = clip_limit
#         self.tile_grid_size = tile_grid_size
    
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X):
#         """
#         Apply CLAHE to enhance the contrast of images.
#         """
#         clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
#         return [clahe.apply(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in X]

# def preprocess_images(image_folder, output_dir):
#     """
#     Preprocess images by cropping and applying CLAHE, then save the processed images.
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     processed_dir = os.path.join(output_dir, 'processed_images(Clache+size)')
#     if not os.path.exists(processed_dir):
#         os.makedirs(processed_dir)
    
#     # Create the pipeline
#     pipeline = make_pipeline(
#         EggShapeCropper(width=1930, height=1000, axes_ratio=(1, 1.2)),  # Adjust the axes_ratio to control the shape
#         CLAHEEnhancer(clip_limit=2.0, tile_grid_size=(16, 16))
#     )
    
#     # Get a list of image files in the folder
#     image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder)]
    
#     for image_path in image_files:
#         try:
#             img = cv2.imread(image_path, cv2.IMREAD_COLOR)
#             if img is not None:
#                 # Apply the pipeline
#                 processed_img = pipeline.transform([img])[0]
                
#                 output_path = os.path.join(processed_dir, os.path.basename(image_path))
#                 cv2.imwrite(output_path, processed_img)
#                 logging.info(f"Processed and saved image: {output_path}")
#             else:
#                 logging.error(f"Failed to load image: {image_path}")
#         except Exception as e:
#             logging.error(f"Error processing image {image_path}: {e}")

# # Folder containing your X-ray images (adjust this path)
# image_folder_path = r"C:\Users\hp\Desktop\2914---1470"
# # Directory where you want to save processed images
# processed_images_dir = r"C:\Users\hp\Desktop\ProcessedImages"

# # Call the function with the image folder path and processed directory
# preprocess_images(image_folder_path, processed_images_dir)


2024-08-18 13:05:20,935 - INFO - Processed and saved image: C:\Users\hp\Desktop\ProcessedImages\processed_images(Clache+size)\000000021972_NESE OZMEN_Panorama_20170627134309.jpg
2024-08-18 13:05:20,978 - INFO - Processed and saved image: C:\Users\hp\Desktop\ProcessedImages\processed_images(Clache+size)\000000021977_ABDULKADIR SAGCAN_Panorama_20170627163926.jpg
2024-08-18 13:05:21,042 - INFO - Processed and saved image: C:\Users\hp\Desktop\ProcessedImages\processed_images(Clache+size)\0000005045_AYSE HEKIMOGLU_Panorama_20140730133721.jpg
2024-08-18 13:05:21,093 - INFO - Processed and saved image: C:\Users\hp\Desktop\ProcessedImages\processed_images(Clache+size)\1.3.6.1.4.1.25403.193662124755707.11904.20240112140657.136.jpg
2024-08-18 13:05:21,143 - INFO - Processed and saved image: C:\Users\hp\Desktop\ProcessedImages\processed_images(Clache+size)\1.3.6.1.4.1.25403.193662124755707.1340.20240320104501.55.jpg
2024-08-18 13:05:21,193 - INFO - Processed and saved image: C:\Users\hp\Desktop\P